- [문제 링크](https://www.acmicpc.net/problem/2206)

In [52]:
import sys
from collections import deque
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline

# (1,1) -> (N, M)까지 이동해야 한다 : (0, 0) -> (N-1, M-1)로 ㄱ
# 단 벽을 부술 기회가 1번 있다
# 최단 경로를 구하시오
N, M = map(int, input().split())
matrix = []
for _ in range(N):
    matrix.append(list(map(int, list(input().strip()))))

dq = deque([]) # 큐의 각 원소는 [현재 위치, 경로, 벽 부수기 가능 여부]
# visited = [[0, 0] * M for _ in range(N)]

# 방문 리스트를 3차원으로 구성한다
visited = [[[0] * 2 for _ in range(M)] for _ in range(N)]

moves = [(1, 0), (-1, 0), (0, 1), (0, -1)]
def bfs(start_loc):
    
    count = 1
    breakable = 1 # 부술 수 있다
    dq.append((start_loc, count, breakable))
    visited[start_loc[0]][start_loc[1]][breakable] = 1
    
    while dq:
        now_loc, now_count, now_breakable = dq.popleft()
        
        if now_loc == (N-1, M-1):
            return now_count
        
        for move in moves:
            drow = now_loc[0] + move[0]
            dcol = now_loc[1] + move[1]
            
            # 아래 조건은 이런 식으로 쓸 수 있다. 머쓱
            # if 0 <= da < n and 0 <= db < m:
            if drow < 0 or drow >= N:
                continue
            if dcol < 0 or dcol >= M:
                continue
            if now_breakable == 0 and matrix[drow][dcol] == 1:
                continue
            
            if visited[drow][dcol][now_breakable] == 0:
                
                if now_breakable == 1 and matrix[drow][dcol] == 1:
                    dq.append(((drow, dcol), now_count + 1, 0))
                    visited[drow][dcol][0] = 1
                    
                elif matrix[drow][dcol] == 0:
                    dq.append(((drow, dcol), now_count + 1, now_breakable))
                    visited[drow][dcol][now_breakable] = 1
    
    # 큐를 다 돌렸는데 함수가 끝나지 않았다
    # = 목적지에 도착하지 못했다
    return -1

print(bfs((0, 0)))

15


In [51]:
for i in range(N):
    print(visited[i])

[[1, 1], [1, 1], [1, 1], [1, 1]]
[[1, 0], [1, 0], [1, 0], [1, 1]]
[[1, 1], [1, 1], [1, 1], [1, 1]]
[[1, 1], [1, 0], [1, 0], [1, 0]]
[[1, 0], [0, 0], [0, 0], [0, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0]]


### 반례 찾기 : 19%에서 자꾸 빠꾸남
- 아 이런 케이스가 있다
```
0000
1110
0000
0111
1111
0000
```
```
처음부터 세로로 달리면 벽을 하나 뚫는데, 그 다음 벽에서 막힘
반면 돌아서 가면 벽을 뚫을 기회가 남아있기 때문에 도착점에 도착할 수 있음
근데 원래 구현한 알고리즘("깊이" 차원이 없는 지도)에 의하면 이미 0번째 column들을 방문한 적이 있기 때문에 돌아온 애들은 접근할 수가 없음 (`-1`출력)
```
- 따라서 **방문 배열을 다르게 구성할 필요가 있음**
    - 지금 같은 경우 벽을 뚫었느냐 여부로 결정했기 때문에
    - 벽을 뚫었을 때의 방문 리스트와 벽을 뚫지 않았을 때의 방문 리스트를 따로 구성하면 됨
- 그렇게 되면 벽을 뚫든 뚫지 않았든 관계 없이 마지막 점에 도달했다면 큐에서 popleft되기 때문에 끝나고
- 큐가 다 돌았는데도 함수가 끝나지 않았다면 그건 목적지에 도착하는 경로가 없다는 의미이므로 -1